In [2]:
import pandas as pd

# 项目路径
project_path = '/home/wjunneng/Python/2019-Iflytek-Mobile-Advertising-Anti-Fraud-Algorithm-Challenge'
# round1_iflyad_anticheat_testdata_feature.txt文件路径
testdata_feature_path = project_path + '/data/original/round1_iflyad_anticheat_testdata_feature.txt'
# round1_iflyad_anticheat_traindata.txt文件路径
traindata_path = project_path + '/data/original/round1_iflyad_anticheat_traindata.txt'

def get_testdata_feature(**params):
    """
    返回 testdata_feature 文件内容
    :param params:
    :return:
    """
    testdata_feature_data = pd.read_table(testdata_feature_path, sep='\t')

    return testdata_feature_data


def get_traindata(**params):
    """
    返回 traindata 文件内容
    :param params:
    :return:
    """
    traindata_data = pd.read_table(traindata_path, sep='\t')

    return traindata_data


testdata_feature = get_testdata_feature()
traindata = get_traindata()
traindata.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 29 columns):
sid             1000000 non-null object
label           1000000 non-null int64
pkgname         1000000 non-null object
ver             678176 non-null object
adunitshowid    1000000 non-null object
mediashowid     1000000 non-null object
apptype         1000000 non-null float64
nginxtime       1000000 non-null float64
ip              1000000 non-null object
city            984807 non-null object
province        1000000 non-null float64
reqrealip       1000000 non-null object
adidmd5         1000000 non-null object
imeimd5         1000000 non-null object
idfamd5         1000000 non-null object
openudidmd5     1000000 non-null object
macmd5          1000000 non-null object
dvctype         1000000 non-null float64
model           995533 non-null object
make            876161 non-null object
ntt             1000000 non-null float64
carrier         1000000 non-null float64
os     

In [ ]:
traindata.columns



In [ ]:
# 包名 经过MD5码 加密 【有重复值，有空值，建议one-hot】
print(traindata['pkgname'].unique())
print('\n')

# app版本 【很杂，建议剔除】
print(traindata['ver'].describe())
print('\n')

# 对外广告位 （广告显示的位置）【有重复值，无空值，建议one-hot】
print(traindata['adunitshowid'].describe())
print('\n')

# 对外媒体ID  【有重复值，无空值，建议one-hot】
print(traindata['mediashowid'].describe())
print('\n')

# app所属分类  【有重复值，无空值，建议one-hot】
print(traindata['apptype'].describe())
print('\n')




In [ ]:
# 请求到达服务时间
print(traindata['nginxtime'].unique())
print(traindata['nginxtime'].loc[0:3])
print('\n')




In [ ]:
# 客户端IP地址
print(traindata['ip'].unique())
print('\n')




In [22]:
###############################################################################
# 城市
print(traindata['city'].unique())
print('\n')

# 省份
print(traindata['province'].unique())
print('\n')


def deal_city_province(df, **params):
    """
    处理城市和省份
    :param df: 
    :param params: 
    :return: 
    """
    import json
    # 中国省市编码地址
    china_city_list_path = project_path + '/data/foreign/china_city_list.json'
    
    # 省
    provinces = {}
    
    # 市
    cities = {}

    # 读取json
    with open(china_city_list_path, 'r') as file:
        # 读取所有省市数据
        china_city_list = json.load(file)
            
        for province in china_city_list:
            # key:province/value:code
            provinces[province['province']] = province['code']
            
            for city in province['cities']:
                # key:city/value:code
                cities[city['name']] = city['code']
    
    china_city_list = df['city']
    china_city_list = china_city_list.fillna(0)
    
    china_city_list_code = []
    china_province_list_code = []
    
    # 遍历获取code
    for city in china_city_list.values:
        # 是否在provinces中
        if city in dict.fromkeys(provinces.keys(), True):
            tmp = provinces[city]
            china_city_list_code.append(tmp)
            china_province_list_code.append(tmp[:2])
            
        # 是否在cities中
        elif city in dict.fromkeys(cities.keys(), True):
            tmp = cities[city]
            china_city_list_code.append(tmp)
            china_province_list_code.append(tmp[:2])
        
        # 不存在    
        else:
            china_city_list_code.append('100000')
            china_province_list_code.append('10')
            
            if city != 0:
                print(city)
    
    # 替换
    df['city'] = china_city_list_code
    df['province'] = china_province_list_code
                
    return df
    
traindata = deal_city_province(traindata)

# 城市
print(traindata['city'].unique())
print('\n')

# 省份
print(traindata['province'].unique())
print('\n')
###############################################################################




['石家庄市' '开封市' '长春市' '曲靖市' '深圳市' '潍坊市' '宜宾市' '邯郸市' '贵阳市' '来宾市' '盐城市' '南京市'
 '宁波市' '重庆市' '泰州市' '常州市' '西安市' '商丘市' '广州市' '连云港市' '唐山市' '北京市' '安阳市' '新乡市'
 '武汉市' '青岛市' nan '眉山市' '甘孜藏族自治州' '江门市' '沈阳市' '东莞市' '桂林市' '厦门市' '吕梁市'
 '哈尔滨市' '成都市' '内江市' '晋中市' '贵港市' '凉山彝族自治州' '徐州市' '衢州市' '福州市' '宿州市' '珠海市'
 '绍兴市' '昆明市' '西宁市' '上饶市' '朔州市' '天津市' '上海市' '枣庄市' '赣州市' '南宁市' '锡林郭勒盟' '黄石市'
 '昌吉回族自治州' '郑州市' '贺州市' '杭州市' '丽水市' '郴州市' '衡阳市' '汕尾市' '泰安市' '钦州市' '惠州市'
 '怀化市' '淮安市' '日照市' '清远市' '湘潭市' '河源市' '临沂市' '朝阳市' '南通市' '保定市' '秦皇岛市'
 '呼和浩特市' '吉安市' '巴音郭楞蒙古自治州' '合肥市' '周口市' '牡丹江市' '邢台市' '长沙市' '遂宁市' '湛江市'
 '洛阳市' '大兴安岭地区' '兰州市' '九江市' '济南市' '葫芦岛市' '荆门市' '滨州市' '驻马店市' '沧州市' '龙岩市'
 '南阳市' '济宁市' '乌兰察布市' '丽江市' '南昌市' '攀枝花市' '文山壮族苗族自治州' '延边朝鲜族自治州' '揭阳市'
 '黔南布依族苗族自治州' '信阳市' '鄂尔多斯市' '通辽市' '铜川市' '无锡市' '池州市' '阜新市' '百色市' '通化市'
 '东营市' '孝感市' '湖州市' '锦州市' '荆州市' '平顶山市' '阜阳市' '伊春市' '温州市' '廊坊市' '梧州市' '肇庆市'
 '海口市' '莆田市' '庆阳市' '丹东市' '佳木斯市' '常德市' '泉州市' '长治市' '金昌市' '乌鲁木齐市' '淄博市'
 '鹰潭市' '太原市' '黔西南布依族苗族自治州' '芜湖市' '金华市' '铁岭市' '焦作市' '咸阳市' '淮南市' '遵义市' '四平

In [ ]:
# 请求的http协议头携带IP，有可能是下游服务器的ip
print(traindata['reqrealip'].unique())
print('\n')




In [ ]:
# Adroid ID的MD5值
print(traindata['adidmd5'].unique())
print('\n')

# imei的MD5值  移动设备国际识别码，是手机设备的唯一识别号码
print(traindata['imeimd5'].unique())
print('\n')

# idfa的MD5值
print(traindata['idfamd5'].unique())
print('\n')

# openudid的MD5值
print(traindata['openudidmd5'].unique())
print('\n')

# mac的MD5值
print(traindata['macmd5'].unique())
print('\n')

# 设备类型 0 – 未知,1 – PC,2 – 手机, 3– 平板,4– 电视盒,5– 智能电视,6 – 可穿戴设备,7 – 智能家电,8 - 音箱,9 - 智能硬件
print(traindata['dvctype'].unique())
print('\n')

# 机型
print(traindata['model'].unique())
print('\n')




In [8]:
# 厂商
print(traindata['make'].value_counts())
print('\n')

# 网络类型 0-未知, 1-有线网, 2-WIFI, 3-蜂窝网络未知, 4-2G, 5-3G, 6–4G
print(traindata['ntt'].unique())
print('\n')

# 运营商 0-未知, 46000-移动, 46001-联通, 46003-电信
print(traindata['carrier'].unique())
print('\n')



OPPO                    250359
vivo                    207547
HUAWEI                  126195
Xiaomi                   48005
HONOR                    45321
                         ...  
ONEPLUS+A5010                1
OPPO A53m                    1
K-Touch K5                   1
%2522vivo%2BX9s%2522         1
%2522STF-AL00%2522           1
Name: make, Length: 2620, dtype: int64




In [ ]:
###############################################################################
# 操作系统 : Android, iOS
print(traindata['os'].unique())
print('\n')

# 操作系统 
print(testdata_feature['os'].unique())
print('\n')

def deal_os(df, **params):
    """
    处理操作系统
    :param df: 
    :param params: 
    :return: 
    """
    if 'os' in df.columns:
        # 先将所有值转化为小写
        df['os'] = df['os'].apply(lambda x: x.lower())
        # 如果所有值都相同，则剔除
        if len(set(df['os'])) is 1:
            del df['os']
    
    return df
    
traindata = deal_os(traindata)

# 操作系统版本
print(traindata.columns)
print('\n')

testdata_feature = deal_os(testdata_feature)

# 操作系统版本
print(testdata_feature.columns)
print('\n')
###############################################################################




In [ ]:
# 操作系统版本
print(traindata['osv'].unique())
print('\n')




In [ ]:
###############################################################################
# 横竖屏:0竖屏，1横屏
print(traindata['orientation'].unique())
print('\n')

# 横竖屏:0竖屏，1横屏
print(testdata_feature['orientation'].unique())
print('\n')

# train 中orientation为90的仅有一条数据为0，其余结为1 

# (1, 28)
train_2 = traindata[traindata.orientation.astype(int) == 2]
print(train_2.shape)
# (0, 27)
test_2 = testdata_feature[traindata.orientation.astype(int) == 2] 
print(test_2.shape)
# (110, 28)
train_90 = traindata[traindata.orientation.astype(int) == 90] 
print(train_90.shape)
# (15, 27)
test_90 = testdata_feature[traindata.orientation.astype(int) == 90] 
print(test_90.shape)

def deal_orientation(df, **params):
    """
    剔除非0和1的数据 【即非横/竖屏的数据】
    :param df: 
    :param params: 
    :return: 
    """
    df = df[(df['orientation'].astype(int) == 0) | (df['orientation'].astype(int) == 1) | (df['orientation'].astype(int) == 90)]
    
    return df

traindata = deal_orientation(traindata)
testdata_feature = deal_orientation(testdata_feature)

# 横竖屏:0竖屏，1横屏
print(traindata['orientation'].unique())
print('\n')

# 横竖屏:0竖屏，1横屏
print(testdata_feature['orientation'].unique())
print('\n')
###############################################################################




In [ ]:
###############################################################################
# 语言
print(traindata['lan'].value_counts())
print('\n')

print(testdata_feature['lan'].value_counts())
print('\n')

# print(traindata[traindata['lan'] == 'en'][['label', 'province', 'city']])

def deal_lan(df, **params):
    """
    处理语言
    :param df: 
    :param params: 
    :return: 
    """
    df['lan'] = df['lan'].fillna(0)
    
    df['lan'] = df['lan'].apply(lambda x: str(x).lower())
    # 优先处理cn: 1
    df['lan'] = df['lan'].apply(lambda x: '1' if 'cn' in str(x) else x)
    # 处理tw： 2
    df['lan'] = df['lan'].apply(lambda x: '2' if 'tw' in str(x) else x)
    # 处理hk: 2
    df['lan'] = df['lan'].apply(lambda x: '2' if 'hk' in str(x) else x)
    # 处理en: 3
    df['lan'] = df['lan'].apply(lambda x: '3' if 'en' in str(x) else x)
    # 处理zh: 1
    df['lan'] = df['lan'].apply(lambda x: '1' if 'zh' in str(x) else x)
    # 其他： 4
    df['lan'] = df['lan'].apply(lambda x: '4' if not x in ['0', '1', '2', '3'] else x)
    
    return df

# 处理语言
traindata = deal_lan(traindata)
testdata_feature = testdata_feature(testdata_feature)

print(traindata['lan'].value_counts())
print('\n')

print(testdata_feature['lan'].value_counts())
print('\n')
###############################################################################




In [ ]:
# 设备高
print(traindata['h'].unique())
print('\n')

# 设备宽
print(traindata['w'].unique())
print('\n')

# 屏幕密度
print(traindata['ppi'].unique())
print('\n')